In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23208710
paper_name = 'lis_bobek_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/AAC.01439-12_zac999101582so1.xlsx', sheet_name='all data')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5902 x 5


In [7]:
original_data.head()

,ORF,MUC7,HSN,KR20,hLF
0,YAL001C,0.074574,-0.105230,-0.054383,0.157555
1,YAL002W,-0.561490,-0.775043,-0.612000,-0.089684
2,YAL003W,-0.327110,0.114625,0.491980,-0.437597
3,YAL004W,-0.495468,-0.076517,0.248498,-0.413881
4,YAL005C,0.421255,0.446952,-0.285837,-0.134814


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, MUC7, HSN, KR20, hLF, orf]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [13]:
original_data = -original_data[['MUC7','HSN','KR20','hLF']].copy()

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(5880, 4)

In [16]:
original_data.head()

,MUC7,HSN,KR20,hLF
orf,,,,
YAL001C,-0.074574,0.105230,0.054383,-0.157555
YAL002W,0.561490,0.775043,0.612000,0.089684
YAL003W,0.327110,-0.114625,-0.491980,0.437597
YAL004W,0.495468,0.076517,-0.248498,0.413881
YAL005C,-0.421255,-0.446952,0.285837,0.134814


### Split essentials and non-essentials

In [17]:
essential_orfs = original_data.index.values[is_essential(original_data.index.values)]
nonessential_orfs = original_data.index.values[~is_essential(original_data.index.values)]

Loading SGD features from 2017-04-03.
Loading SGD features from 2017-04-03.


In [18]:
original_data1 = original_data.loc[nonessential_orfs,:].copy()
original_data2 = original_data.loc[essential_orfs,:].copy()

In [19]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [20]:
original_data.head()

,MUC7_1,HSN_1,KR20_1,hLF_1,MUC7_2,HSN_2,KR20_2,hLF_2
orf,,,,,,,,
YAL001C,NaN,NaN,NaN,NaN,-0.074574,0.105230,0.054383,-0.157555
YAL002W,0.561490,0.775043,0.612000,0.089684,NaN,NaN,NaN,NaN
YAL003W,NaN,NaN,NaN,NaN,0.327110,-0.114625,-0.491980,0.437597
YAL004W,0.495468,0.076517,-0.248498,0.413881,NaN,NaN,NaN,NaN
YAL005C,-0.421255,-0.446952,0.285837,0.134814,NaN,NaN,NaN,NaN


# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [516, 4925, 4926, 4927, 11870, 11871, 11872, 11873]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,516,4925,4926,4927,11870,11871,11872,11873
data_type,value,value,value,value,value,value,value,value
orf,,,,,,,,
YAL001C,NaN,NaN,NaN,NaN,-0.074574,0.105230,0.054383,-0.157555
YAL002W,0.561490,0.775043,0.612000,0.089684,NaN,NaN,NaN,NaN
YAL003W,NaN,NaN,NaN,NaN,0.327110,-0.114625,-0.491980,0.437597
YAL004W,0.495468,0.076517,-0.248498,0.413881,NaN,NaN,NaN,NaN
YAL005C,-0.421255,-0.446952,0.285837,0.134814,NaN,NaN,NaN,NaN


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 17


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,516,4925,4926,4927,11870,11871,11872,11873
,data_type,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,
1,YAL001C,NaN,NaN,NaN,NaN,-0.074574,0.105230,0.054383,-0.157555
2,YAL002W,0.561490,0.775043,0.612000,0.089684,NaN,NaN,NaN,NaN
3,YAL003W,NaN,NaN,NaN,NaN,0.327110,-0.114625,-0.491980,0.437597
1863,YAL004W,0.495468,0.076517,-0.248498,0.413881,NaN,NaN,NaN,NaN
4,YAL005C,-0.421255,-0.446952,0.285837,0.134814,NaN,NaN,NaN,NaN


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,516,4925,4926,4927,11870,11871,11872,11873,516,4925,4926,4927,11870,11871,11872,11873
,data_type,value,value,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,
1,YAL001C,NaN,NaN,NaN,NaN,-0.074574,0.105230,0.054383,-0.157555,NaN,NaN,NaN,NaN,0.085318,0.445106,0.300967,-0.124179
2,YAL002W,0.561490,0.775043,0.612000,0.089684,NaN,NaN,NaN,NaN,1.111800,1.753536,1.401812,0.180677,NaN,NaN,NaN,NaN
3,YAL003W,NaN,NaN,NaN,NaN,0.327110,-0.114625,-0.491980,0.437597,NaN,NaN,NaN,NaN,0.975592,-0.022357,-1.045859,1.375838
1863,YAL004W,0.495468,0.076517,-0.248498,0.413881,NaN,NaN,NaN,NaN,0.962443,0.077289,-0.451402,0.821742,NaN,NaN,NaN,NaN
4,YAL005C,-0.421255,-0.446952,0.285837,0.134814,NaN,NaN,NaN,NaN,-1.111417,-1.178874,0.699369,0.269917,NaN,NaN,NaN,NaN


# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/8 [00:00<?, ?it/s]

Deleting all datasets for PMID 23208710...
Inserting the new data...


100%|██████████| 8/8 [01:08<00:00,  8.52s/it]

Updating the data_modified_on field...
